In [ ]:
############ CARREGA TODAS AS LINHAS 2.0

import pandas as pd
colunas=['COD_CONTA','PRODUTO','NOME','CPF_CNPJ','SENHA']
# Caminho para o arquivo CSV
file_path = r"R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\11\Jorgin da Clock\BASE DE COBRANÇA INTERSIC\BASE_COBRANCA_20230806_ATN_154/BASE_COBRANCA_20230806_ATN_154.txt"

base_claro = pd.read_csv(file_path, sep="\t", dtype=str, encoding='iso-8859-1', engine='python', error_bad_lines=False, warn_bad_lines=True,skiprows=1,usecols = colunas)

base_claro = base_claro.query("PRODUTO == 'Claro MÃ³vel'")
base_claro['COD_CONTA'] = base_claro['COD_CONTA'].astype('int64')

base_claro = base_claro[base_claro.apply(lambda x: len(str(x["COD_CONTA"])) == 9, axis=1)]
base_claro = base_claro.reset_index(drop=True)


base_claro


In [ ]:
base_claro.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\11\Jorgin da Clock\BASE PDD 1108/base_claro_localizada_1108.csv",index=False,sep=";")

In [ ]:
# carregando as bases de pdd que normalmente sao dividida em 2
pdd_1 = pd.read_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Setembro\8\Markin 7\BASE_PPD_0809\Base_Pre_PDD_1/Base_Pre_PDD_1.txt",names=['ACCOUNT_ID'])
pdd_2 = pd.read_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Setembro\8\Markin 7\BASE_PPD_0809\Base_Pre_PDD_2/Base_Pre_PDD_2.txt",names=['ACCOUNT_ID'])

pdd = pd.concat([pdd_1,pdd_2])

pdd = pdd.drop_duplicates(['ACCOUNT_ID']).reset_index(drop=True)
pdd

In [ ]:
# fazendo um merges com os groupbys de cada DF
base_claro = pd.merge(pdd, base_claro,  right_on=["COD_CONTA"], left_on=["ACCOUNT_ID"], how="inner")
base_claro

In [ ]:
# CARREGANDO A BASE COM O TELEFONE HIGIENIZADO E CONTRATANTE
df_mailing = pd.read_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO/BASE_OPERACIONAL_TELECOB_HUGO.txt",sep=';',dtype=str)
df_mailing = df_mailing.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','TELEFONE_HIGIENIZADO','ATRASO','BASE']].query("BASE == 'Claro Móvel'")
df_mailing

In [ ]:
base_claro = pd.merge(base_claro, df_mailing,  right_on=["DEVEDOR"], left_on=["SENHA"], how="inner")\
    .loc[:,['ACCOUNT_ID','COD_CONTA','PRODUTO','CPF_CNPJ','NOME','SENHA','TELEFONE','TELEFONE_HIGIENIZADO','BASE','ATRASO']]\
        .drop_duplicates(['ACCOUNT_ID','COD_CONTA','CPF_CNPJ','NOME','SENHA','TELEFONE','TELEFONE_HIGIENIZADO'])


base_contratante = base_claro.loc[:,['ACCOUNT_ID','COD_CONTA','PRODUTO','CPF_CNPJ','NOME','SENHA','TELEFONE','BASE','ATRASO']]
base_higienizada = base_claro.loc[:,['ACCOUNT_ID','COD_CONTA','PRODUTO','CPF_CNPJ','NOME','SENHA','TELEFONE_HIGIENIZADO','BASE','ATRASO']]
base_higienizada = base_higienizada.rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})


base_final = pd.concat([base_contratante,base_higienizada])
base_final = base_final.drop_duplicates(['ACCOUNT_ID','COD_CONTA','CPF_CNPJ','NOME','SENHA','TELEFONE'])
base_final = base_final.query("TELEFONE.notnull()")

# ### Baixando se caso precise subir um mailing com essa base 
base_final.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Setembro\8\Markin 7\BASE_PPD_0809/mailing_pdd_1108.csv",index=False,sep=';')



In [ ]:
# resumo de cpf por atraso, jessica sempre pede
group_pdd = base_claro.groupby(['ATRASO'])['CPF_CNPJ'].count().reset_index().sort_values(['ATRASO'],ascending=True)
group_pdd.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Setembro\8\Markin 7\BASE_PPD_0809/group_pdd.csv",index=False,sep=';')

group_pdd